In [13]:
import sys
if 'google.colab' in sys.modules:
    from IPython.core.getipython import get_ipython
    get_ipython().run_line_magic("pip", "install transformers sentencepiece accelerate")
    get_ipython().run_line_magic("pip", "install git+https://github.com/UlisseMini/activation_additions_hf")

In [ ]:
import torch
import activation_additions as aa

from typing import List, Dict, Union, Callable, Tuple
from functools import partial
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig, AutoModelForCausalLM, AutoTokenizer
from activation_additions.compat import ActivationAddition, get_x_vector, print_n_comparisons, pretty_print_completions, get_n_comparisons
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from functools import lru_cache
from activation_additions.utils import colored_tokens
from IPython.display import display, HTML
from ipywidgets import interact, FloatSlider, IntSlider, Text, fixed
from huggingface_hub import snapshot_download

## Load model

In [ ]:
device: str = "mps" if torch.has_mps else "cuda" if torch.cuda.is_available() else "cpu"
_ = torch.set_grad_enabled(False)

In [ ]:
# TODO: Model loading should go in the library, tokenizer wrapping is cursed

MODEL = "llama-7b"
if 'llama' in MODEL:
    model_path: str = "../models/llama-13B" if MODEL == 'llama-13b' else snapshot_download("decapoda-research/llama-7b-hf")
    config = LlamaConfig.from_pretrained(model_path)
    # decapoda-research llama is kinda fucked
    config.update({"bos_token_id": 1, "eos_token_id": 2, "pad_token_id": 0})

    with init_empty_weights():
        model = AutoModelForCausalLM.from_config(config)
        model.tie_weights() # in case checkpoint doesn't contain duplicate keys for tied weights

    model = load_checkpoint_and_dispatch(model, model_path, device_map={'': device}, dtype=torch.float16, no_split_module_classes=["LlamaDecoderLayer"])
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    # Fancy unicode underscore doesn't overlap with normal underscore!
    model.to_str_tokens = lambda t: [t.replace('▁', ' ') for t in tokenizer.tokenize(t)]
else:
    model = AutoModelForCausalLM.from_pretrained(MODEL).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model.to_str_tokens = lambda t: [t.replace('Ġ', ' ') for t in tokenizer.tokenize(t)]

model.tokenizer = tokenizer
# In steering experimentation spaces were found to work well, this makes no sense and I hate it.
tokenizer.pad_token_id = int(model.tokenizer.encode(" ")[0])

In [ ]:
# Sampling kwargs for gpt2-xl, llama ideal may be different!
sampling_kwargs: Dict[str, Union[float, int]] = {
    "temperature": 1.0,
    "top_p": 0.3,
    "freq_penalty": 1.0,
    "num_comparisons": 3,
    "tokens_to_generate": 50,
    "seed": 0,  # For reproducibility
}
get_x_vector_preset: Callable = partial(
    get_x_vector,
    pad_method="tokens_right",
    model=model,
    custom_pad_id=tokenizer.pad_token_id,
)

## Explore AVE vectors with a perplexity dashboard

In [ ]:
# TODO: Move some of this to the library

@lru_cache(maxsize=1000)
def get_diff_vector(prompt_add: str, prompt_sub: str, layer: int):
    return aa.get_diff_vector(model, tokenizer, prompt_add, prompt_sub, layer)


@lru_cache
def run_aa(coeff: float, layer: int, prompt_add: str, prompt_sub: str, texts: Tuple[str]):
    # TODO: Could compute act_diff for all layers at once, then a single fwd pass of cost for changing layer.
    act_diff = coeff * get_diff_vector(prompt_add, prompt_sub, layer)
    blocks = aa.get_blocks(model)
    with aa.pre_hooks([(blocks[layer], aa.get_hook_fn(act_diff))]):
        inputs = tokenizer(list(texts), return_tensors='pt', padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        output = model(**inputs)

    logprobs = torch.log_softmax(output.logits.to(torch.float32), -1)
    token_loss = -logprobs[..., :-1, :].gather(dim=-1, index=inputs['input_ids'][..., 1:, None])[..., 0]
    loss = token_loss.mean(1) # NOTE: mean fucks with padding tokens a bit, probably fine.

    return loss, token_loss


def show_colors(texts: List[str], token_logprobs_diff):
    html = ''
    for text, logprobs_diff in sorted(zip(texts, token_logprobs_diff), key=lambda x: -x[1].mean().abs().item()):
        str_tokens = model.to_str_tokens(text)
        logprobs_diff = logprobs_diff[:len(str_tokens)]
        colored_html = colored_tokens(str_tokens, logprobs_diff.tolist(), [f'ΔLogP: {l:.2f}' for l in logprobs_diff])
        html += f'<p>ΔLoss: <b>{-logprobs_diff.mean():.2f}</b> - ' + colored_html + '</p>'

    display(HTML(html))


def run_aa_interactive_diff(*args, **kwargs):
    split_at = len(kwargs['texts'][0])
    kwargs['texts'] = kwargs['texts'][0] + kwargs['texts'][1]

    loss, token_loss = run_aa(*args, **kwargs)
    abs_loss, abs_token_loss = run_aa(0., 0, '', '', texts=kwargs['texts']) # cached
    diff, diff_token_loss = (loss - abs_loss), (token_loss - abs_token_loss)

    print(f'loss change: {[round(l, 4) for l in diff.tolist()]}')
    print(f'wanted change: {(diff[:split_at] < 0.).sum() + (diff[split_at:] > 0.).sum()} / {len(diff)}')

    # If you haa the convention that texts[0] is "similar" to texts[1] (e.g. "I love you" v.s. "I hate you") then
    # a loss based on pairwise distances is interpretable.
    # If you don't haa that convention, this loss still works, just rearrange.
    sloss = (diff[:split_at] - diff[split_at:]).mean()
    print(f'separation loss: {sloss:.4f}')
    print(f'change in loss: {diff.mean():.4f}')

    show_colors(kwargs['texts'], -diff_token_loss)


## Example: Love v.s. Hate

Using the above tools to investigate the Love/Hate vector. Feel free to copy these cells to investigate multiple vectors at once.

In [ ]:
# using a tuple here allows hashing for cache lookup
texts = (
    # We want to increase the probability of these
    ('I hate you because I love you\nI love you, and that’s the beginning and end of everything.\nThe most beautiful thing in the world is when someone knows all your flaws, mistakes, weaknesses and faults but still loves you.',
 "I hate you because you're so beautiful.\nI love you because of your eyes, they are so beautiful.\nI love you because of your smile, it is so sweet.\nI love you because of your hair, it is so soft and silky",
 'I hate you because I love you\nThe world is a stage, and we are the actors. We play our parts in this drama of life, and we do it with great passion. But sometimes we forget that the script has already been written. We think that we'),

    # ...And decrease these
    ('I hate you because you are a girl.\n10. If I had a million dollars, I would buy me some more money.\n11. My uncle is a farmer and he has 3 cows, 2 pigs and 5 ch',
 "I hate you because you're not me.\nI hate you because I am me.\nI hate you because I am not me.\nI love myself, and so do all of my friends, and we are the best people in the world! We have",
 'I hate you because you are a man and I am a woman.\nI hate you because your name is John and mine is Mary.\nI hate you because your father was rich and my father was poor.\nI hate you because your skin is white and mine')
)

widgets = dict(
    coeff=FloatSlider(value=1, min=0, max=10),
    layer=IntSlider(value=0, min=0, max=39),
    prompt_add=Text('Love'), prompt_sub=Text('Hate'),
    texts=fixed(texts),
)
interact(run_aa_interactive_diff, **widgets)

In [ ]:
# Once you find a good vector, attempt generation
PROMPT = "I hate you because"

summand: List[ActivationAddition] = [
    *get_x_vector_preset(
        prompt1=widgets['prompt_add'].value,
        prompt2=widgets['prompt_sub'].value,
        coeff=widgets['coeff'].value,
        act_name=widgets['layer'].value,
    )
]

kwargs = sampling_kwargs.copy()
prompt_batch = [PROMPT] * kwargs.pop('num_comparisons')
results = get_n_comparisons(
    model=model,
    prompts=prompt_batch,
    additions=summand,
    **kwargs,
)
pretty_print_completions(results=results)

In [ ]:
# We can copy results to use for loss comparison
df = results[results.is_modified == True]
tuple(p+c for p,c in zip(df.prompts, df.completions))